# Fix a few large outliers on FAR / DUA in the zoning lookups

In [5]:
import pandas as pd
import pathlib
import geopandas as gpd
import os

In [33]:
m_drive = pathlib.Path("/Volumes/Data/Models") if os.name != "nt" else "M:/"
m_drive

PosixPath('/Volumes/Data/Models')

In [31]:
# p10_topfix_path = m_drive / "Data" / "GIS layers" / "JobsHousingTransitProximity" / "update_2024" / "outputs" / "p10_topofix_classified.parquet"
# p10_topfix = gpd.read_parquet(p10_topfix_path)

In [35]:
# set paths

zoning_lookup_path = m_drive / "urban_modeling" / "baus" / "BAUS Inputs" / \
    "basis_inputs" / "zoning" / "2024_04_09_zoning_lookup_hybrid_he_MS_pba50plus.csv"

zoning_parcels_path = m_drive / "urban_modeling" / "baus" / "BAUS Inputs" / \
    "basis_inputs" / "zoning" / "2024_04_09_zoning_parcels_hybrid_he_MS_pba50plus.csv"

In [36]:
# load parcels
zoning_parcels = pd.read_csv(zoning_parcels_path)

In [37]:
# load lookups
zoning_lookup = pd.read_csv(zoning_lookup_path)

# add a tracker field
zoning_lookup['adjusted'] = None

In [38]:
# Check for the largest outliers wrt FAR
large_far_mask = zoning_lookup.eval('max_far>35')
print(large_far_mask.value_counts())

# overwrite to missing
zoning_lookup.loc[large_far_mask, ['max_far', 'adjusted']] = None, 'Clip >35'

False    17913
True         4
Name: count, dtype: int64


In [39]:
large_dua_mask = zoning_lookup.eval('max_dua>500')
print(large_dua_mask.value_counts())

# overwrite to missing
zoning_lookup.loc[large_dua_mask, ['max_dua', 'adjusted']] = None, 'Clip >500'

False    17863
True        54
Name: count, dtype: int64


In [40]:
# large dev in pleasanton is for specific zone id 10199
large_dev_mask = zoning_lookup.eval('id==10199')

print(large_dev_mask.value_counts())

zoning_lookup.loc[large_dev_mask, ['max_dua',
                                   'adjusted']] = None, 'Set to 10 from 154'

False    17916
True         1
Name: count, dtype: int64


In [41]:
# set path for new zoning lookup table
zoning_lookup_path_new = m_drive / "urban_modeling" / "baus" / "BAUS Inputs" / "basis_inputs" / "zoning" / "2024_04_09_zoning_lookup_hybrid_he_MSClip_pba50plus.csv"

In [ ]:
zoning_lookup.to_csv(zoning_lookup_path_new)

In [42]:
zoning_lookup[zoning_lookup.id==10199]

,id,juris,name,max_dua,max_far,max_height,HS,HT,HM,OF,...,SC,IL,IW,IH,RS,RB,MR,MT,ME,adjusted
9199,10199,pleasanton,PUD-RDR/OS,NaN,NaN,NaN,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,Set to 10 from 154
